# RSA Encryption Project

This is an implemenation of RSA encryption to encrypt and decrypt files between clients
The first person generates public keys which they send to the second party and they use those public keys to encrypt a message/file and a decryption key to use later for decryption. They send the public key to another party who can use it to encrypt a message/file. The encrypted message is outputted to the file "encrypted-file". They can then send the file over to the first person, who holds a decryption key, which they can use to decrypt the file and view the message.

In [1]:
import random
import time
import math
import numpy as np
import matplotlib.pyplot as plt

RSA relies on generating large prime numbers and values related to them which is the bulk of this project. There is no known "formula" to cook up prime numbers, instead, the only way to find prime numbers is to pick a random number and test for primality, which is still a hard task.

In [2]:
# Miller Rabin is a probabilstic test for primality. Works with "75% chance". This function runs the test 100 times on each number
# Returns False if the number is definitely composite
# Returns True if the number is probably prime
def miller_rabin(n):
    """
    Miller-Rabin primality test.
 
    A return value of False means n is certainly not prime. A return value of
    True means n is very likely a prime.
    """
    if n!=int(n):
        return False
    n=int(n)
    # Check small values first
    if n==0 or n==1 or n==4 or n==6 or n==8 or n==9:
        return False
 
    if n==2 or n==3 or n==5 or n==7:
        return True
    s = 0
    d = n-1
    while d%2==0:
        d>>=1
        s+=1
    assert(2**s * d == n-1)
    
    # Try to prove compositness and return False
    def trial_composite(a):
        if pow(a, d, n) == 1:
            return False
        for i in range(s):
            if pow(a, 2**i * d, n) == n-1:
                return False
        return True  
    
    # Try for 1 <= a <= 100 
    for i in range(100):#number of trials 
        a = random.randrange(2, n)
        if trial_composite(a):
            return False
 
    return True  

In [3]:
# Recursive Implementation of the Extended Euclidean Algorithm
def eea(a, b):
    '''
    solves for u,v in au+bv=gcd(a,b)
    returns g: gcd(a,b), u, v 
    '''
    if a == 0: #base step
        return (b, 0, 1)
    else: #recursive step
        gcd, u, v = eea(b % a, a)
    # Divide and round down
    return gcd, v - (b // a) * u, u

In [4]:
# Helper Function to generate public and private keys
def generate_keys():
    """
    Generate public and private keys N, e, d, phi
    """
    p = 0
    q = 0
    tried = set([]) 
    
    """
    Generate two prime numbers
    Generate random odd numbers between two random numbers
    Checks if the number has been checked before, if not, checks if it is prime
    return the number if it is, otherwise adds the number to the set of checked values and tries again
    """
    isPPrime = miller_rabin(p)
    isQPrime = miller_rabin(q)
    
    while not isPPrime :
        tried.add(p)
        p = random.randrange(10000000000000000001, 100000000000000000001, 2) #random odd number on the magnitude of 10^20
        if(p not in tried): # Check only if the number hasn't been tried before
            isPPrime = miller_rabin(p)
        else: # If this number has already been checked before, look for a new number
            tried.add(p)
            p = random.randrange(10000000000000000001, 100000000000000000001, 2) #random odd number, not huge, but good enough for this project
            
    while not isQPrime :
        tried.add(q)
        q = random.randrange(10000000000000000001, 100000000000000000001, 2) #random odd number on the magnitude of 10^20
        if(q not in tried): # Check only if the number hasn't been tried before
            isQPrime = miller_rabin(q)
        else: # If this number has already been checked before, look for a new number
            tried.add(q)
            q = random.randrange(10000000000000000001, 100000000000000000001, 2) #random odd number on the magnitude of 10^20
    N = p*q
    phi = (p-1)*(q-1)
    
    
    # Generate the encryption exponent e
    e = random.randrange(1000001, 10000001)
    gcd, d, k = eea(e, phi)
    while(gcd != 1):
        # Notice, since the primes we generate are all odd, phi has to be even
        # So we need only look for odd number values for e
        e = random.randrange(100000000001, 10000000000001, 2)
        gcd, d, k = eea(e, phi)
    
    # d might be negative, we can simply add phi of n as it doesn't affect it's value modulo phi
    if (d < 0):
        d+= phi
    return N, e, d

In [5]:
# Function to encrypt a message
def encrypt(message, N, e): 
    message_array = list(message) # Converts to a list of characters
    message_ascii_arr = []
    encrypted = []
    for i in range(len(message)): # I had some problems with converting between str and ints so I did this with a for loop
        message_ascii_arr.append(ord(message[i]))
        encrypted.append(pow(message_ascii_arr[i], e, N))
    ecnrypted_text = ','.join(map(str, encrypted))
    out_file = open("encrypted-file", "w")
    out_file.truncate(0)
    out_file.write(ecnrypted_text)
    out_file.close()
    print("\n Succesfully encrypted to encrypted-file \n")

In [6]:
# Function to decrypt a message
def decrypt(encrypted, N, d):
    decrypted = ""
    encrypted_arr = []
    encrypted_str_arr = []
    encrypted_str_arr = encrypted.split(",")
    encrypted_arr = list(map(int, encrypted_str_arr))
    for i in range(len(encrypted_arr)):
        ascii_val = pow(encrypted_arr[i], d, N)
        decrypted += chr(ascii_val)
    decrypted_file = open("decrypted-file", 'w')
    decrypted_file.truncate(0)
    decrypted_file.write(decrypted)
    decrypted_file.close()
    print("\n Succesfully decrypted to decrypted-file \n")

In [7]:
if __name__ == "__main__":
    # Driver Code
    print("Welcome! this is an Encryptor that uses RSA to encrypt files/messages. Here's how to use it:")
    print("First you need to generate the public keys e and N and private key d. Publish/send the public")
    print("keys e and N to anyone you want. They can use them to encrypt the files and send them to you.")
    print("Then you can recover the the original message. \n")
    choice = "a"
    generated = False
    while(choice != "q"):
        print("What do you want to do")
        print("(G)enerate public and private keys")
        print("(E)ncrypt files")
        print("(D)ecrypt files")
        print("(Q)uit")
        choice = (input("")).lower()

        if choice == "g":
            password = ""
            while password == "":
                password = input("Please enter a password to use for decryption: ")
            N, e, d = generate_keys()
            generated = True
            print("N: ", N, "\t e: ", e, "\n")


        elif choice == "e":
            decision = (input("Use (G)enerated public keys or (D)ifferent values(Will override previous values): ")).lower()
            if decision == "g" and generated == True :
                print("Using N: ", N, "\t e: ", e, "\n")
            elif decision == "g" and generated == False :
                raise Exception("No public keys generated yet")
            elif decision == "d" : 
                N = input("Enter value for N: ")
                e = input("Enter value for e: ")
            else:
                raise Exception("Invalid Input!")

            decision = (input("use a (S)tring or (F)ile: ")).lower()
            if decision == "s" :
                m = input("Type Message: ")
            elif decision == "f" :
                decision = input("Type the name of the file")
                file = open(decision, "w")
                try:
                    m = file.read()
                except:
                    print("An exception ocurred. File not readable")
            else : 
                raise Exception("Invalid Input!")
            encrypt(m, N, e)
        elif choice == "d":
            if not generated: 
                raise Exception("No decryption keys generated")
            else:
                decision = ""
                while decision != password :
                    decision = input("Enter Password: ")
                encrypted_file = open("encrypted-file", 'r')
                encrypted_text = encrypted_file.read()
                decrypt(encrypted_text, N, d)

Welcome! this is an Encryptor that uses RSA to encrypt files/messages. Here's how to use it:
First you need to generate the public keys e and N and private key d. Publish/send the public
keys e and N to anyone you want. They can use them to encrypt the files and send them to you.
Then you can recover the the original message. 

What do you want to do
(G)enerate public and private keys
(E)ncrypt files
(D)ecrypt files
(Q)uit
g
Please enter a password to use for decryption: 1234
N:  4339328301592214929532170464960088112279 	 e:  9292957 

What do you want to do
(G)enerate public and private keys
(E)ncrypt files
(D)ecrypt files
(Q)uit
e
Use (G)enerated public keys or (D)ifferent values(Will override previous values): g
Using N:  4339328301592214929532170464960088112279 	 e:  9292957 

use a (S)tring or (F)ile: s
Type Message: I need sleep

 Succesfully encrypted to encrypted-file 

What do you want to do
(G)enerate public and private keys
(E)ncrypt files
(D)ecrypt files
(Q)uit
d
Enter Pass